# Getting Started:
## A simple driving model training and evaluation pipeline using the Drive360 dataset and PyTorch.

## Loading data from Drive360 dataset.

The **dataset.py** file contains the 3 classes necessary for creating a Drive360Loader. Using the **config.json** file to specify the location of the csv and data directory, we can generate phase (train, validation, test) specific data loaders that can output samples from each set. Adjust the **dataset.py** to your preferred training framework.

In [ ]:
import json
from dataset import Drive360Loader

# load the config.json file that specifies data 
# location parameters and other hyperparameters 
# required.
config = json.load(open('./config.json'))

# create a train, validation and test data loader
train_loader = Drive360Loader(config, 'train')
validation_loader = Drive360Loader(config, 'validation')
test_loader = Drive360Loader(config, 'test')

# print the data (keys) available for use. See full 
# description of each data type in the documents.
print('Loaded train loader with the following data available as a dict.')
print(train_loader.drive360.dataframe.keys())



Phase: train # of data: 233352
Phase: validation # of data: 106219
Phase: test # of data: 279863
Loaded train loader with the following data available as a dict.
Index(['cameraRight', 'cameraFront', 'cameraRear', 'cameraLeft', 'here',
       'tomtom', 'gpsLatitude', 'gpsLongitude', 'gpsAltitude', 'gpsPrecision',
       'hereMmLatitude', 'hereMmLongitude', 'hereSpeedLimit',
       'hereSpeedLimit_2', 'hereFreeFlowSpeed', 'hereSignal', 'hereYield',
       'herePedestrian', 'hereIntersection', 'hereMmIntersection',
       'hereSegmentExitHeading', 'hereSegmentEntryHeading',
       'hereSegmentOthersHeading', 'hereCurvature', 'hereCurrentHeading',
       'here1mHeading', 'here5mHeading', 'here10mHeading', 'here20mHeading',
       'here50mHeading', 'hereTurnNumber', 'canSteering', 'canSpeed',
       'chapter', 'bin_canSteering'],
      dtype='object')


## Training a basic driving model

Create your driving model. This is specific to your learning framework. 

Below we give a very basic dummy model that uses the front facing camera and a resnet34 + LSTM architecture to predict canSteering and canSpeed. 

In [8]:
from torchvision import models
import torch.nn as nn
import torch

class SomeDrivingModel(nn.Module):
    def __init__(self):
        super(SomeDrivingModel, self).__init__()
        final_concat_size = 0
        
        # Main CNN
        cnn = models.resnet34(pretrained=True)
        self.features = nn.Sequential(*list(cnn.children())[:-1])
        self.intermediate = nn.Sequential(nn.Linear(
                          cnn.fc.in_features, 128),
                          nn.ReLU())
        final_concat_size += 128

        # Main LSTM
        self.lstm = nn.LSTM(input_size=128,
                            hidden_size=64,
                            num_layers=3,
                            batch_first=False)
        final_concat_size += 64
        
        # Angle Regressor
        self.control_angle = nn.Sequential(
            nn.Linear(final_concat_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        # Speed Regressor
        self.control_speed = nn.Sequential(
            nn.Linear(final_concat_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    
    def forward(self, data):
        module_outputs = []
        lstm_i = []
        # Loop through temporal sequence of
        # front facing camera images and pass 
        # through the cnn.
        for k, v in data['cameraFront'].items():
            x = self.features(v)
            x = x.view(x.size(0), -1)
            x = self.intermediate(x)
            lstm_i.append(x)
            # feed the current front facing camera
            # output directly into the 
            # regression networks.
            if k == 0:
                module_outputs.append(x)

        # Feed temporal outputs of CNN into LSTM
        i_lstm, _ = self.lstm(torch.stack(lstm_i))
        module_outputs.append(i_lstm[-1])
        
        # Concatenate current image CNN output 
        # and LSTM output.
        x_cat = torch.cat(module_outputs, dim=-1)
        
        # Feed concatenated outputs into the 
        # regession networks.
        prediction = {'canSteering': torch.squeeze(self.control_angle(x_cat)),
                      'canSpeed': torch.squeeze(self.control_speed(x_cat))}
        return prediction

# Create your own driving model, this is
#  a very basic one. 
model = SomeDrivingModel()




A basic training procedure that iterates over the train_loader and feeds each sample into our dummy model, subsequently calculates loss. We kill after 20 batches just

In [9]:
import torch.optim as optim

criterion = nn.SmoothL1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
model.train()
for epoch in range(1):
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        prediction = model(data)
        # Ony optimizing for canSpeed at the moment
        # add canSteering to optimize simulatenously.
        loss = criterion(prediction['canSpeed'], target['canSpeed'])
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if batch_idx % 2 == 1:  
            print('[epoch: %d, batch:  %5d] loss: %.5f' %
                  (epoch + 1, batch_idx + 1, running_loss / 2.0))
            running_loss = 0.0
        # Remove this when actually training. 
        # Used to terminate early. 
        if batch_idx >= 20: 
            break
            
            



FileNotFoundError: Traceback (most recent call last):
  File "/scratch_net/hispalensis/heckers/Applications/anaconda2/envs/pytorch-1.1/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 99, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/scratch_net/hispalensis/heckers/Applications/anaconda2/envs/pytorch-1.1/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 99, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/heckers/PycharmProjects/learn-to-drive-starter-kit/dataset.py", line 258, in __getitem__
    if self.front:
  File "/scratch_net/hispalensis/heckers/Applications/anaconda2/envs/pytorch-1.1/lib/python3.6/site-packages/PIL/Image.py", line 2652, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: './data/Emmen/go_pro_4/image/0/img01525.jpg'


## Local evaluation of the model.

In [ ]:
import numpy as np
model.eval()
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(validation_loader):
        prediction = model(data)
        # Again only evaluating the canSpeed 
        # predictions, add canSteering when 
        # jointly training.
        mse = (np.square(prediction['canSpeed'] - 
                    target['canSpeed'])).mean()
        print(mse)
        # Used to terminate early, remove.
        if batch_idx >= 5: 
            break


## Creating a submission file.

In [ ]:
normalize_targets = config['target']['normalize']
target_mean = config['target']['mean']
target_std = config['target']['std']

def add_results(results, output):
    steering = np.squeeze(output['canSteering'].cpu().data.numpy())
    speed = np.squeeze(output['canSpeed'].cpu().data.numpy())
    if normalize_targets:
        steering = (steering*target_std['canSteering'])+target_mean['canSteering']
        speed = (speed*target_std['canSpeed'])+target_mean['canSpeed']
    if np.isscalar(steering):
        steering = [steering]
    if np.isscalar(speed):
        speed = [speed]
    results['canSteering'].extend(steering)
    results['canSpeed'].extend(speed)


We use pandas to create a submission file which is simply a 2-column csv with a canSteering and canSpeed prediction for each row in the **drive360_test.csv** a total of 305437 rows/predictions not including the header. See the **sample_submission.csv** file as an example.

IMPORTANT: for the test phase indices will start 10s (100 samples) into each chapter this is to allow challenge participants to experiment with different temporal settings of data input. If challenge participants have a greater temporal length than 10s for each training sample, then they must write a custom function here. Please check out the **dataset.py** file for additional explanation.

In [ ]:
import pandas as pd

file = './submission.csv'
results = {'canSteering': [],
           'canSpeed': []}
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
        prediction = model(data)
        add_results(results, prediction)

        # Used to terminate early, remove.
        if batch_idx >= 5: 
            break
        
df = pd.DataFrame.from_dict(results)
df.to_csv(file, index=False)